In [2]:
import numpy as np
import pandas as pd
import requests
import json
import dotenv
import os

## Getting Biographical Data from the Congress API

The most important skill is reading the API documentation efficiently.

Look ONLY for these five things:

1. Root: the beginning part of all URLs associated with this API for getting data back (I often find this in examples first)

    https://api.congress.gov/v3


2. Endpoint: the second part of the URL, generally it points to a specific dataset to be returned

    /member/{bioguideId}

    The curly braces mean: fill this in for whichever member of congress you want


3. The headers: data that we give to the API to identify outselves, and provide other information if requested
    
    * User Agent string: a string that identifies your software. If you are writing a bot, what is your bot's name, and what methods are you using to access this API.

    The standard convention is: 
    `botname/version (email) python-requests/request-package-version`

In [3]:
botname = 'targ'
version = '0.0'
email = ''
useragent = f'{botname}/{version} ({email}) python-requests/{requests.__version__}'
headers = {'User-Agent': useragent}

4. API Parameters: these are ways to control what data is requested. The parameters are generally listed in the documentation for each endpoint. 3 kinds of parameters:

    * Query parameters: should be listed in a dictionary and passed to the `params` argument of `requests.get()`

    * Path parameters: are part of the endpoint (such as bioguideId)

    * Global parameters: apply to any use of the API regardless of endpoint; often includes your API key; can be in either the headers or the params

5. If an API Key is needed, how to get one, and how will you let the API know the key?

    API keys should be saved in a .env file in the same folder as your code. Use `dotenv` package to load the key into your code without ever displaying/exposing your keys.

In [4]:
dotenv.load_dotenv()
congresskey = os.getenv("congresskey")
params = {'format': 'json',
          'api_key': congresskey}

Put it all together.

In [5]:
bioguideId = 'M001239'
root = 'https://api.congress.gov/v3'
endpoint = f'/member/{bioguideId}'

r = requests.get(root + endpoint, 
                 headers = headers, 
                 params = params)

In [6]:
r.json()

{'member': {'addressInformation': {'city': 'Washington',
   'district': 'DC',
   'officeAddress': '1013 Longworth House Office Building',
   'phoneNumber': '(202) 225-4711',
   'zipCode': 20515},
  'bioguideId': 'M001239',
  'birthYear': '1968',
  'cosponsoredLegislation': {'count': 168,
   'url': 'https://api.congress.gov/v3/member/M001239/cosponsored-legislation'},
  'currentMember': True,
  'depiction': {'attribution': 'Image courtesy of the Member',
   'imageUrl': 'https://www.congress.gov/img/member/67744ba20b34857ecc909149_200.jpg'},
  'directOrderName': 'John J. McGuire III',
  'district': 5,
  'firstName': 'John',
  'honorificName': 'Mr.',
  'invertedOrderName': 'McGuire, John J.',
  'lastName': 'McGuire',
  'middleName': 'J.',
  'officialWebsiteUrl': 'https://mcguire.house.gov/',
  'partyHistory': [{'partyAbbreviation': 'R',
    'partyName': 'Republican',
    'startYear': 2025}],
  'previousNames': [{'directOrderName': 'John McGuire',
    'endDate': '2025-01-10T17:36:18Z',
   

### Sponsored legislation

In [7]:
endpoint = f'/member/{bioguideId}/sponsored-legislation'
endpoint

'/member/M001239/sponsored-legislation'

In [9]:
params = {'format': 'json',
          'offset': 0,
          'limit': 250,
          'api_key': congresskey}

In [10]:
r = requests.get(root + endpoint, 
                 headers=headers, 
                 params=params)
r

<Response [200]>

In [13]:
r.json()

{'pagination': {'count': 9},
 'request': {'bioguideId': 'm001239',
  'contentType': 'application/json',
  'format': 'json'},
 'sponsoredLegislation': [{'congress': 119,
   'introducedDate': '2025-09-26',
   'latestAction': {'actionDate': '2025-09-26',
    'text': 'Referred to the House Committee on Ways and Means.'},
   'number': '5595',
   'policyArea': {'name': None},
   'title': 'To amend the Internal Revenue Code of 1986 to modify the tax on remittance transfers.',
   'type': 'HR',
   'url': 'https://api.congress.gov/v3/bill/119/hr/5595?format=json'},
  {'congress': 119,
   'introducedDate': '2025-09-19',
   'latestAction': {'actionDate': '2025-09-19',
    'text': 'Referred to the House Committee on Armed Services.'},
   'number': '5522',
   'policyArea': {'name': None},
   'title': 'To change the name of the Department of Defense to the "Department of War", and for other purposes.',
   'type': 'HR',
   'url': 'https://api.congress.gov/v3/bill/119/hr/5522?format=json'},
  {'congres

In [16]:
myjson = r.json()
sponsoredLegislation_df = pd.json_normalize(myjson, record_path=['sponsoredLegislation'])

In [17]:
for i in sponsoredLegislation_df['title']:
    print(i)

To amend the Internal Revenue Code of 1986 to modify the tax on remittance transfers.
To change the name of the Department of Defense to the "Department of War", and for other purposes.
Border Wall Status Act
Make the District of Columbia Safe and Beautiful Act
FAIR Act
VA Data Transparency and Trust Act
Manned Aircraft Clarification Act
Uranium for Energy Independence Act of 2025
Agricultural and Forestry Hauling Efficiency Act


In [20]:
r = requests.get(sponsoredLegislation_df['url'][1], 
                 headers=headers, 
                 params=params)
r.json()

{'bill': {'actions': {'count': 3,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5522/actions?format=json'},
  'committees': {'count': 1,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5522/committees?format=json'},
  'congress': 119,
  'constitutionalAuthorityStatementText': '<pre>\n[Congressional Record Volume 171, Number 154 (Friday, September 19, 2025)]\n[House]\nFrom the Congressional Record Online through the Government Publishing Office [<a href="https://www.gpo.gov">www.gpo.gov</a>]\nBy Mr. McGUIRE:\nH.R. 5522.\nCongress has the power to enact this legislation pursuant\nto the following:\nArticle I, Section 8\n[Page H4489]\n</pre>',
  'introducedDate': '2025-09-19',
  'latestAction': {'actionDate': '2025-09-19',
   'text': 'Referred to the House Committee on Armed Services.'},
  'legislationUrl': 'https://www.congress.gov/bill/119th-congress/house-bill/5522',
  'number': '5522',
  'originChamber': 'House',
  'originChamberCode': 'H',
  'sponsors': [{'bioguideId': 'M0012

### FEC Data for financial campaign contributions

In [21]:
dotenv.load_dotenv()
feckey = os.getenv("feckey")

In [23]:
root = 'https://api.open.fec.gov'
endpoint = '/v1/candidates/search/'

params = {'api_key': feckey,
          'q': 'John McGuire',
          'state': 'VA',
          'district': '5',
          'office': 'H'}

r = requests.get(root + endpoint, 
                 headers=headers, 
                 params=params)
r

<Response [200]>

In [27]:
myjson = r.json()
candidate_id = myjson['results'][0]['candidate_id']
candidate_id

'H0VA07133'

In [ ]:
# three John McGuire

root = 'https://api.open.fec.gov'
endpoint = '/v1/names/candidates/'

params = {'api_key': feckey,
          'q': 'John McGuire'}

r = requests.get(root + endpoint, 
                 headers=headers, 
                 params=params)

r.json()

{'api_version': '1.0',
 'results': [{'id': 'H0VA07133',
   'name': 'MCGUIRE, JOHN',
   'office_sought': 'H'},
  {'id': 'H2VA07196', 'name': 'MCGUIRE, JOHN', 'office_sought': 'H'},
  {'id': 'H0FL16029', 'name': 'MCGUIRE, JOHN MICHAEL', 'office_sought': 'H'}]}